This notebook presents the semi-physical model of the L1 laser in its second version, based on PyTorch.

Author: Francesco Capuano, 2022 S17 summer intern @ ELI-beamlines, Prague


# Motivation

The goal of this project is to maximise second-harmonic efficiency. However, since it is also very much related to the shortest possible pulse shape, we started with developing a strategy to optimise a predefinite set of control parameters so as to minimise the difference between the obtained pulse shape (in the temporal domain) and a target one (which, by default, is the shortest one typically). 

However, since data are really expensive to empirically collect we resorted to model the underlying dynamics of the whole system, also considering that (even if not exhaustive) there is a significant amount of know-how concerned with the considered dynamics available.
This knowledge about the actual physical process is presented in the following Figure. 

![](img/semi-physical_model.png)

The very same diagram can also be represented by means of a **computational graph**, presented in the following Figure: 

![](img/semiphysicalmodel-computational_graph.png)

This visualisation shows how the sequence of operations that do lead to the final temporal shape is the **succession of much more elementary operations**. 

This is particularly useful when auto-differentiation can be deployed. In this particular setting because, if one represents the semiphysical model as a computational graph, then it is possible to access, very cheaply in terms of computational time and effort (and with high analytical precision), to those **differential information** which are crucial to **apply methods such as Newton Method or any gradient-based method**.

Surely enough, such a model is practically applicable only under the assumption that $y_3(\nu)$ is a *good approximation* of $y_{REAL, 3}(\nu)$. 

Luckily enough, various are the frameworks supporting auto-differentiation. One of this is **Pytorch**. Using it basically requires to rewrite the majority of the code present in the initial version of the semi-physical model, considering the data abstraction of this framework, i.e. **tensors** instead of **arrays**. 

In [1]:
# imports and data acquisition
import numpy as np
import torch
from typing import Tuple
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt

# these import are necessary to import modules from directories one level back in the folder structure
import sys
sys.path.append("../")

import os
import pandas as pd
from scipy.constants import c

from utils.LaserModel import LaserModel
data_path = "../data/L1_pump_spectrum.csv"

# read the data
df = pd.read_csv(data_path, header = None)
df.columns = ["Wavelength (nm)", "Intensity"]
# converting Wavelength in Frequency
df["Frequency (THz)"] = df["Wavelength (nm)"].apply(lambda wavelenght: 1e-12 * (c/(wavelenght * 1e-9)))
# clipping everything that is negative
df["Intensity"] = df["Intensity"].apply(lambda intensity: np.clip(intensity, a_min = 0, a_max = None))
# the observations must be returned for increasing values of frequency
df = df.sort_values(by = "Frequency (THz)")

frequency, intensity = df.loc[:, "Frequency (THz)"].values, df.loc[:, "Intensity"].values
intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min())
field = np.sqrt(intensity)

Once the data are ingested it is possible to elaborate them as usual (i.e., using already defined methods) since no differential information about this step is necessary.

In [2]:
from utils.physics import *
# preprocessing
cutoff = np.array((289.95, 291.91)) * 1e12
# cutting off the signal
frequency_clean, field_clean = cutoff_signal(frequency_cutoff = cutoff, frequency = frequency * 1e12,
                                             signal = field)
# augmenting the signal
frequency_clean_aug, field_clean_aug = equidistant_points(frequency = frequency_clean,
                                                          signal = field_clean,
                                                          num_points = int(5e4)) # n_points defaults to 5e3
# retrieving central carrier
central_carrier = central_frequency(frequency = frequency_clean_aug, signal = field_clean_aug)

In [11]:
frequency, field = torch.from_numpy(frequency_clean_aug).cuda(), torch.from_numpy(field_clean_aug).cuda()

AssertionError: Torch not compiled with CUDA enabled

In [5]:
def phase_equation(frequency:torch.tensor, central_frequency:float, GDD:float, TOD:float, FOD:float) -> torch.tensor: 
    """This function returns the phase with respect to the frequency and some control parameters. The various multiplicative
    constants present in phase's formula derive mainly from the necessity of converting everything to SI uom.

    Args:
        frequency (torch.tensor): Tensor of frequencies considered (measured in Hz)
        central_frequency (float): Central frequency, not angular, measured in Hz.
        GDD (float): Group Delay Dispersion, measured in 10^{-30} s^2 (femtoseconds squared).
        TOD (float): Control parameter 1, measured in {10^-45} s^3 (femtoseconds cubed). 
        FOD (float): Control parameter 1, measured in {10^-60} s^4 (femtoseconds to the fourth). 

    Returns:
        torch.tensor: The phase with respect to the frequency, measured in radiants.
    """
    phase = \
            (1/2)* GDD * (2*np.pi * (frequency - central_frequency))**2 + \
            (1/6)* TOD * (2*np.pi * (frequency - central_frequency))**3 + \
            (1/24)* FOD * (2*np.pi * (frequency - central_frequency))**4
    return phase

def mse(x:torch.tensor, y:torch.tensor)->float: 
    """This function computes the MSE between x and y.

    Args:
        x (torch.tensor): First input array
        y (torch.tensor): Second input array

    Returns:
        float: MSE value.
    """
    x, y = x.reshape(-1,), y.reshape(-1,)
    return ((x-y)**2).mean()

def temporal_profile(frequency:torch.tensor, field:torch.tensor, phase:torch.tensor, npoints_pad:int=int(1e4), return_time:bool=True) -> Tuple[np.array, np.array]:
    """This function returns the temporal profile of a given signal considering the signal itself (in the frequency domain) and a given phase. 
    Padding is added so as to have more points and increase FFT algorithm output's quality. 

    Args:
        frequency (torch.tensor): Array of frequencies considered (measured in Hz)
        field (torch.tensor): Array of field measured in the frequency domain. 
        phase (torch.tensor): Array representing the phase considered in the frequency domain. 
        npoints_pad (int, optional): Number of points to be used in padding. Padding will be applied using half of this value on the
        right and half on the left. Defaults to int(1e4).
        return_time (bool, optional): Whether or not to return also the time frame of the signal to be used on the x-axis. Defaults to True. 
    Returns:
        Tuple[np.array, np.array]: Returns either (time, intensity) (with time measured in in femtoseconds) or intensity only.
    """
    step = torch.diff(frequency)[0]
    time = torch.fft.fftshift(torch.fft.fftfreq(len(frequency) + npoints_pad, d=abs(step)))
    
    field_padded = torch.nn.functional.pad(field,
                                           pad = (npoints_pad // 2, npoints_pad // 2),
                                           mode = "constant",
                                           value = 0)
    phase_padded = torch.nn.functional.pad(phase,
                                           pad = (npoints_pad // 2, npoints_pad // 2),
                                           mode = "constant",
                                           value = 0)

    field_time = torch.fft.fftshift(torch.fft.fft(field_padded * torch.exp(1j * phase_padded))) # inverse FFT to go from frequency domain to temporal domain
    intensity_time = torch.real(field_time * torch.conj(field_time)) # only for casting reasons

    intensity_time = intensity_time / intensity_time.max() # normalizing
    
    # either returning time or not according to return_time
    if not return_time: 
        return intensity_time
    else: 
        return time, intensity_time